# Heat Map of Countries and their Mean Rank

In [1]:
import geopandas as gpd
import pandas as pd
import folium
import os
import time
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service

# Load the GeoJSON as a GeoDataFrame
world_geojson = 'https://raw.githubusercontent.com/johan/world.geo.json/master/countries.geo.json'
world = gpd.read_file(world_geojson)
# Load in SAP data
data = pd.read_csv('data/final_rankings.csv')

In [2]:
# Hotfix

mistakes = ['Egypt, Arab Rep.',
             'Brunei Darussalam',
             'Congo, Dem. Rep.',
             'Iran, Islamic Rep.',
             "Cote d'Ivoire",
             'North Macedonia',
             "Korea, Dem. People's Rep.",
             'Congo, Rep.',
             'Russian Federation',
             "Korea, Rep.",
             "Viet Nam",
             "Turkiye",
             "Serbia",
             "Tanzania",
             "Czechia",
             "Syrian Arab Republic",
             "United States",
             "Venezuela, RB",
             "Kyrgyz Republic",
             "Lao PDR",
             "Yemen, Rep.",
             "Slovak Republic",
             "Bahamas, The"]

fix = ['Egypt', 
       'Brunei',
       'Democratic Republic of the Congo',
       'Iran',
       'Ivory Coast',
       'Macedonia',
       'North Korea',
       'Republic of the Congo',
       'Russia',
       'South Korea',
       "Vietnam",
       "Turkey",
       "Republic of Serbia",
       "United Republic of Tanzania",
       "Czech Republic",
       "Syria",
       "United States of America",
       "Venezuela",
       "Kyrgyzstan",
       "Laos",
       "Yemen",
       "Slovakia",
       "The Bahamas"]


for i in range(len(mistakes)):
       index = data[data['Country Name'] == mistakes[i]].index[0]
       data.loc[index, "Country Name"] = fix[i]

In [3]:
list_of_countries_gp = set(world['name']) 
list_of_countries_sap = set(data['Country Name'])
common_countries = list(list_of_countries_sap.intersection(list_of_countries_gp))

# Filter your dataset to keep only countries in the intersection
data_filtered = data[data['Country Name'].isin(common_countries)]
world_filtered = world[world['name'].isin(common_countries)]

In [4]:
# Create a Folium map centered around the world
m = folium.Map(location=[20, 0], zoom_start=2)

# Add Choropleth layer for visualization
folium.Choropleth(
    geo_data=world_filtered,  # Filtered GeoJSON
    name='choropleth',
    data=data_filtered,  # Filtered dataset
    columns=['Country Name', 'mean_rank'],  # Column to plot
    key_on='feature.properties.name',  # Ensure country names match
    fill_color='Reds',  # Choose color scheme (can change to 'Viridis', 'Reds', etc.)
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Mean Rank Score'
).add_to(m)

# Save the map as an HTML file
m.save("data/map.html")

# Show map
m